## ***important libraries:)***



In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns',None)
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


# ***Exploring the dataset:)***

In [ ]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty_level"]

In [ ]:
train = pd.read_csv('/content/train_COMPETITION.csv',header=None, names=col_names)
test=pd.read_csv('/content/test_COMPETITION.csv')


In [ ]:
display(train.info())
display(test.info())


<class 'pandas.core.frame.DataFrame'>
Index: 17252 entries, ID to 18146
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   duration                     17252 non-null  object
 1   protocol_type                17252 non-null  object
 2   service                      17252 non-null  object
 3   flag                         17252 non-null  object
 4   src_bytes                    17252 non-null  object
 5   dst_bytes                    17252 non-null  object
 6   land                         17252 non-null  object
 7   wrong_fragment               17252 non-null  object
 8   urgent                       17252 non-null  object
 9   hot                          17252 non-null  object
 10  num_failed_logins            17252 non-null  object
 11  logged_in                    17252 non-null  object
 12  num_compromised              17252 non-null  object
 13  root_shell                   17252 

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5751 entries, 0 to 5750
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           5751 non-null   int64  
 1   duration                     5751 non-null   int64  
 2   protocol_type                5751 non-null   object 
 3   service                      5751 non-null   object 
 4   flag                         5751 non-null   object 
 5   src_bytes                    5751 non-null   int64  
 6   dst_bytes                    5751 non-null   int64  
 7   land                         5751 non-null   int64  
 8   wrong_fragment               5751 non-null   int64  
 9   urgent                       5751 non-null   int64  
 10  hot                          5751 non-null   int64  
 11  num_failed_logins            5751 non-null   int64  
 12  logged_in                    5751 non-null   int64  
 13  num_compromised   

None

## ***check messing data and duplicated:)***

In [ ]:

def check_null_duplicate(df):
    nu = df.isnull().sum()
    print('NUMBER OF NULL FOR EVERY COLUMN \n', nu[nu > 0])
    print("----------------------------")
    dub = df.duplicated().sum()
    print('NUMBER OF DUPLICATES FOR EVERY COLUMN \n', dub[dub > 0])
    print("----------------------------")

print("Checking train data :")
check_null_duplicate(train)

print("Checking  test data:")
check_null_duplicate(test)

Checking train data :
NUMBER OF NULL FOR EVERY COLUMN 
 Series([], dtype: int64)
----------------------------
NUMBER OF DUPLICATES FOR EVERY COLUMN 
 []
----------------------------
Checking  test data:
NUMBER OF NULL FOR EVERY COLUMN 
 Series([], dtype: int64)
----------------------------
NUMBER OF DUPLICATES FOR EVERY COLUMN 
 []
----------------------------


# ***transform data:)***

In [ ]:
from sklearn.preprocessing import LabelEncoder
lab=LabelEncoder()
obj=train.select_dtypes(include='object')
non_obj=train.select_dtypes(exclude='object')
ob=test.select_dtypes(include='object')
non_ob=test.select_dtypes(exclude='object')
for i in range(0,obj.shape[1]):
   obj.iloc[:,i]=lab.fit_transform(obj.iloc[:,i])
for i in range(0,obj.shape[1]):
   ob.iloc[:,i]=lab.fit_transform(ob.iloc[:,i])

In [ ]:
test=pd.concat([ob,non_ob],axis=1)
train=pd.concat([obj,non_obj],axis=1)

# ***Split Data:)***

In [ ]:
from sklearn.model_selection import train_test_split
train_x = train.iloc[:, :-1]
train_y = train.iloc[:, -1]
test_x = test.iloc[:, :-1]
test_y = test.iloc[:, -1]
x_train, x_test, y_train, y_test = train_test_split(train_x,train_y ,test_size=0.2, random_state=42)


# **scalingdata:_)**

In [ ]:
from sklearn.preprocessing import RobustScaler,MinMaxScaler
Min=MinMaxScaler()
x_train=Min.fit_transform(x_train)
x_test=Min.transform(x_test)

# ***best parameters by csv model:)***

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, RandomForestRegressor

from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
# Define the parameter grids for each model

param_grid_rf = {'n_estimators': [10, 50, 100, 150, 200], 'max_depth': [1, 2, 3, 4, 5]}
param_grid_svc = {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'kernel': ['linear']}
param_grid_xgb = {'n_estimators': [10, 50, 100, 150, 200], 'max_depth': [1, 2, 3, 4, 5]}


# Create the grid search objects for each model

grid_search_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=5)
grid_search_svc = GridSearchCV(SVC(), param_grid_svc, cv=5)
grid_search_xgb = GridSearchCV(XGBClassifier(random_state=42), param_grid_xgb, cv=5)


# Fit the grid search objects to the data

grid_search_rf.fit(x_train, y_train)
grid_search_svc.fit(x_train, y_train)
grid_search_xgb.fit(x_train, y_train)


# Print the best parameters and scores for each model
print("Random Forest - best parameters:", grid_search_rf.best_params_)
print("Random Forest - best score:", grid_search_rf.best_score_)
print("Support Vector Machine - best parameters:", grid_search_svc.best_params_)
print("Support Vector Machine - best score:", grid_search_svc.best_score_)
print("XGBoost - best parameters:", grid_search_xgb.best_params_)
print("XGBoost - best score:", grid_search_xgb.best_score_)


Random Forest - best parameters: {'max_depth': 5, 'n_estimators': 150}
Random Forest - best score: 0.991521739130435
Support Vector Machine - best parameters: {'C': 100, 'kernel': 'linear'}
Support Vector Machine - best score: 0.9810869565217392
XGBoost - best parameters: {'max_depth': 2, 'n_estimators': 200}
XGBoost - best score: 0.9986231884057972


## ***create models:)***

In [ ]:


random_state = 42

models = [
          RandomForestClassifier(max_depth=5, n_estimators=150, random_state=random_state),
          SVC(C=100, kernel='linear', random_state=random_state),
          XGBClassifier(max_depth=2, n_estimators=200, random_state=random_state),
         ]
results = []

for model in models:
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    model_name = type(model).__name__
    results.append([model_name, accuracy])
df = pd.DataFrame(results, columns=['Model', 'Accuracy'])
df = df.sort_values(by=['Accuracy'], ascending=False)

import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(
        x=df['Model'], y=df['Accuracy'],
        text=df['Accuracy'].apply(lambda x: f"{round(x,2)}"),
        textposition='auto',
        offsetgroup=1,
        name='accuracy'
    )
])

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(
    title="Accuracy of Different Models",
    xaxis_title="Model",
    yaxis_title="Accuracy",
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="#3f3f3f"
    )
)

fig.show()



In [ ]:

models = [ RandomForestClassifier(max_depth=5, n_estimators=150, random_state=random_state),
          SVC(C=100, kernel='linear', random_state=random_state),
          XGBClassifier(max_depth=2, n_estimators=200, random_state=random_state),]

best_model = None
best_accuracy = 0

for model in models:
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model

if best_model is not None:
    y_pred = best_model.predict(x_test)
    clas = classification_report(y_test, y_pred)
    co = confusion_matrix(y_test, y_pred)
    labels = np.unique(y_test)
    print(f"Best model is {type(best_model).__name__} with accuracy {best_accuracy:.4f}\n{clas}\n{co}\nLabels: {labels}\n")
else:
    print("No model found.")


Best model is XGBClassifier with accuracy 0.9971
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1808
           1       0.99      1.00      1.00      1643

    accuracy                           1.00      3451
   macro avg       1.00      1.00      1.00      3451
weighted avg       1.00      1.00      1.00      3451

[[1799    9]
 [   1 1642]]
Labels: [0 1]



# ***KMeans:)***

In [ ]:

from sklearn.cluster import KMeans
clu=[]
j=[]
for i in range(1,15):
   km=KMeans(n_clusters=i)
   km.fit(x_train)
   clu.append(i)
   j.append(km.inertia_)
pd.DataFrame(clu,j)


# ***elpow method :)***

In [ ]:
plt.plot(clu,j,marker="o")
plt.title('the best num of KMeans')
plt.xlabel("count of means")
plt.ylabel("error")

plt.show()


In [ ]:
km=KMeans(n_clusters=4)
km.fit(x_train)
pre=km.predict(x_train)

kmea=pd.DataFrame(pre)
df1=kmea[kmea[0]==0]
df1=kmea[kmea[0]==1]
df1=kmea[kmea[0]==2]
df1=kmea[kmea[0]==3]
kmea.value_counts().plot.pie(autopct='%0.2f%%')

In [ ]:
!pip install kneed
from kneed import KneeLocator
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

# create sample data
X, y = make_blobs(n_samples=500, centers=4, random_state=42)

# fit KMeans clustering algorithm for different number of clusters
wcss = []  # within-cluster sum of squares
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=42)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

# plot elbow curve
plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

# find elbow point
elbow_point = KneeLocator(range(1, 11), wcss, curve='convex', direction='decreasing').elbow
print(f"Elbow point is at {elbow_point} clusters.")
